In [1]:
from bs4 import BeautifulSoup, Comment
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from requests import get
import os
import time
import helper

In [2]:
opts = Options()
opts.add_argument(" — headless")
chrome_driver = os.getcwd() +"/../selenium/chromedriver"
print(chrome_driver)
driver = webdriver.Chrome(options=opts, executable_path=chrome_driver)
# Load the HTML page
url = 'https://www.iiitd.ac.in/search/google/policy'
driver.get(url)


/Users/osheensachdev/btp/iiitd_policy_chatbot/notebooks/../selenium/chromedriver


In [3]:
documents = []
for page in range(4):
    page_links = driver.find_elements_by_class_name('gsc-cursor-page')
    page_links[page].click()
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    results = soup.find_all('a', class_ = 'gs-title', href = True)
    documents += list(map(lambda x: {'name': x.get_text(), 'link': x['href']}, results))
documents = {document['link']: document for document in documents}
documents = list(documents.values())

In [4]:
for document in documents:
    if document['link'][-4:] == '.pdf':
        document['format'] = 'pdf'
    else:
        document['format'] = 'html'

In [5]:
for document in documents:
    if document['format'] == 'html':
        driver.get(document['link'])
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        body = soup.find('div', id = 'content-wrap')
        if not body:
            continue
        title = body.find(class_ = 'page-title')
        if not title:
            continue
        print(title.get_text())
        document['title'] = title.get_text()
        document['body'] = str(body)

Academic Dishonesty Policy
Evaluation Policy
Placement Procedure & Policies
Green Policy
Privacy Policy
Student Conduct Policy
Refund / Cancellation Policy
Hostel Policies
Allocation Policies
Internships @ IIIT-D
Internal Forms and Docs
Aasim Khan
Disciplinary Action
Facility Management Services
B.Tech. Fee Waiver
Academic Resources
FAQ about M.Tech. Admissions 2020
M.Tech (CSE & ECE) Admission Details 2019
M.Tech. (CSE & ECE) Admission Details 2020
Rules and Procedures
Fee Schedule for AY 2020-2021
Sumit J Darak
Anubha Gupta
FAQ about M.Tech Admissions 2019
M.Tech. (CSE, ECE and CB)
M.Tech. Assistantship & Studentship
Faculty Recruitment


# Process html files

In [6]:
order = {
    'h0': 0,
    'h1': 1,
    'h2': 2,
    'h3': 3,
    'h4': 4,
    'h5': 5,
    'h6': 6,
    'strong': 10,
    'em': 11,
    'p': 20,
    'a': 20,
    'div': 20,
    'span': 20,
    'ol': 21,
    'li': 21,
}

for document in documents:
    if document['format'] != 'html' or 'body' not in document:
        continue
    body = BeautifulSoup(document['body'], 'html.parser')
    comments = body.findAll(text=lambda text:isinstance(text, Comment))
    for comment in comments:
        comment.extract()

    content = []
    stack = [body]
    elements = body.find_all(text = True)
    for element in elements:
        if str(element) == 'here' or len(str(element).split()) == 0:
            continue
        temp = element
        size = 100
        while temp.parent:
            temp = temp.parent
            if temp.name == 'table' or temp.href:
                size = 100
                break
            if temp.name in order:
                size = min(size, order[temp.name])
        if size == 100:
            continue
        content.append((str(element), size))
        print(size, str(element))
    document['content'] = content

In [7]:
helper.write_json(documents, '../data/files/iiit_website_content.json')